# Busqueda de hiperparametros optimos
Este cuaderno tiene la labor de encontrar la configuracion de hiperparametros optima. Se implementaron estrategias de podado del arbol de decision, sobremuestreo entre otros y validacion cruzada. 

## Importaciones

In [ ]:
import nltk 
import string
import re
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
from progressbar import ProgressBar
import random
from imblearn.over_sampling import RandomOverSampler, SMOTE

## Carga de datos
En este cuaderno solo fue necesario cargar el conjunto de datos de entreno ya que por la estrategia de validacion cruzada se iba a dividir nuevamente este conjunto para verificar que no se estuviera incurriendo en sobreajuste. 

In [2]:
df_train = pd.read_csv("./train.csv")

# Validacion cruzada y busqueda de hiperparametros
Para hallar la configuracion optima del modelo de entrenamiento se establecio la lista de hiperparametros. 


| Hiperparametro | Descripcion |
| --- | --- |
| binary | Elegir si al tokenizar los documentos hacerlo de forma binaria o contando las veces que se repitiera una palabra |
| tfidf | En caso de que no se usara una estrategia binaria se tenia que decidir si usar la estrategia de pesado tfidf |
|oversample|Se tenia que decidir si sobre muestrear la clase minoritaria|
|SMOTE| Se tenia que decidir cual estrategia de sobre muestro realizar si SMOTE o sobre muestrao por bagging con repeticion|
| criterio | el criterio de selecion de variables del arbol podia tomar valores de gini o entropy|
|max_deph | estrategia de podado para evitar crecimiento excesivo del arbol y de esta forma evitar sobre ajuste del modelo se tomaron valores entre 40 y 200|


In [4]:
bar = ProgressBar()
def iterate(train, n = 1000, max_depth_range= (40, 201), n_splits=8):
    best_run = None
    best_average_f1_score = 0
    for i in bar(range(n)):
        run = {}
        
        run["binary"] = random.randint(0,1)
        run["tfidf"] = random.randint(0,1) if not run["binary"] else 0
        run["oversample"] = random.randint(0,1)
        run["SMOTE"] = random.randint(0,1) if run["oversample"] else 0
        run['criterion'] = "gini" if random.randint(0,1) else "entropy"
        run["max_depth"] = random.randrange(max_depth_range[0],max_depth_range[1])
        sum_f1_1 = 0
            
        kf = KFold(n_splits=n_splits)
        kf.get_n_splits(train)
        for train_index, validation_index in kf.split(train):
            y_train = None 
            training_data = train.iloc[train_index]
            validation_data = train.iloc[validation_index]
            
            vect = CountVectorizer(binary=run["binary"])
            vect.fit(training_data["text"])
            train_encoded = vect.transform(training_data["text"])
            validation_encoded = vect.transform(validation_data["text"])
            
            if run["tfidf"]:
                tfidf_transformer = TfidfTransformer()
                tfidf_transformer.fit(train_encoded)
                train_encoded = tfidf_transformer.transform(train_encoded)
                validation_encoded = tfidf_transformer.transform(validation_encoded)

                
            if run['oversample']:
                if run["SMOTE"]:
                    ros = SMOTE()
                else:
                    ros = RandomOverSampler()
                train_encoded, y_train= ros.fit_resample(train_encoded, training_data["spam"])
        
            clf = DecisionTreeClassifier(criterion=run['criterion'], 
                                         max_depth=run['max_depth'],
                                         min_samples_split=2)
        
            if y_train is None:
                y_train = training_data["spam"]
            
            clf.fit(train_encoded, y_train)
            sum_f1_1 += classification_report(validation_data["spam"], 
                                                clf.predict(validation_encoded), output_dict=True)["1"]["f1-score"]
        run["average_f1_score"] = sum_f1_1/n_splits
        
        if best_average_f1_score<run["average_f1_score"]:
            best_run = run
            best_average_f1_score = run["average_f1_score"]
    return best_run
        
best = iterate(df_train)


100% (1000 of 1000) |####################| Elapsed Time: 0:25:06 Time:  0:25:06


Para hallar los hiperparametros adecuados se realizo una busqueda aleatoria en vez de una busqueda en grilla y se itero por 1000 repeticiones, esto tomaba cerca de 20 minutos. Para elegir la configuracion optima se tomo como funcion objetivo el F1-Score. El F1-Score se eligio por las siguientes razones, en el negocio de la mensajeria de texto y multimedia ciertmente es costoso no etiquetar un mensaje fraudulento como fraudulento sin embargo tambien es costoso etiquetar un mensaje como fraudulento sin serlo. Dado que el F1-Score es el promedio entre el sensibilidad y la precision busca balancear estos dos requerimientos de forma que ambos se vean beneficiados. A continuacion se muestra la configuracion optima. Es importante resaltar que la funcion objetivo fue calculada unicamente para lso conjunto de validacion de cada iteracion de validacion cruzada, esto porque el F1-Score para los conjuntos de entreno no es una medida aplicable al mundo real para el modelo ya que se estaria calculando sobre los mismos datos que se entreno. 

In [5]:
best

{'binary': 1,
 'tfidf': 0,
 'oversample': 0,
 'SMOTE': 0,
 'criterion': 'gini',
 'max_depth': 196,
 'average_f1_score': 0.8698598187266303}